In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#printing the stop words
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [6]:
#data preprocessing
dataset=pd.read_csv('/content/drive/MyDrive/train.csv')


In [7]:
dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
dataset.shape

(20800, 5)

0 -------> real
1------->fake

In [9]:
# checking for missing
dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
#replacing the missing value with nul
dataset=dataset.fillna('')

In [11]:
#merging author name and title
dataset['content']=dataset['author'] + ' ' +dataset['title']

In [12]:
dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [13]:
#seperating feature and column
x=dataset.drop(columns='label',axis=1)
y=dataset['label']

In [14]:
print(x)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [15]:
print(y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


stemming:


process of reducing wrds to its keyword or root words

In [31]:
port_stem=PorterStemmer()

In [36]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content='  '.join(stemmed_content)
  return stemmed_content

In [38]:
print(dataset['content'])

0        [darrel, lucu, hous, dem, aid, even, see, come...
1        [daniel, j, flynn, flynn, hillari, clinton, bi...
2            [consortiumnew, com, truth, might, get, fire]
3        [jessica, purkiss, civilian, kill, singl, us, ...
4        [howard, portnoy, iranian, woman, jail, fictio...
                               ...                        
20795    [jerom, hudson, rapper, trump, poster, child, ...
20796    [benjamin, hoffman, n, f, l, playoff, schedul,...
20797    [michael, j, de, la, merc, rachel, abram, maci...
20798    [alex, ansari, nato, russia, hold, parallel, e...
20799                      [david, swanson, keep, f, aliv]
Name: content, Length: 20800, dtype: object


In [22]:
x=dataset['content'].values
y=dataset['label'].values

In [23]:
print(x)

[list(['darrel', 'lucu', 'hous', 'dem', 'aid', 'even', 'see', 'comey', 'letter', 'jason', 'chaffetz', 'tweet'])
 list(['daniel', 'j', 'flynn', 'flynn', 'hillari', 'clinton', 'big', 'woman', 'campu', 'breitbart'])
 list(['consortiumnew', 'com', 'truth', 'might', 'get', 'fire']) ...
 list(['michael', 'j', 'de', 'la', 'merc', 'rachel', 'abram', 'maci', 'said', 'receiv', 'takeov', 'approach', 'hudson', 'bay', 'new', 'york', 'time'])
 list(['alex', 'ansari', 'nato', 'russia', 'hold', 'parallel', 'exercis', 'balkan'])
 list(['david', 'swanson', 'keep', 'f', 'aliv'])]


In [24]:
print(y)

[1 0 1 ... 0 1 1]


In [40]:
y.shape

(20800,)